#  Evaluate the recognition model as whole
## Author: Vilem Gottwald

#### Get features and IDs from objects predicted by the detection algorithm

In [1]:
import numpy as np
from classifier import Ensemble_ovo_ova, FeaturesExtractor
from common import (DATA_PATH, DATASET_SPLIT_IDX, DETECTIONS_PATH, DATASET_PATH,
                    DETECTIONS_GT_CLASSES_PATH, DETECTIONS_IOU_PATH, DETECTED_FEATURES_PATH,
                    DETECTED_OBJECTS_IDS_PATH, COL_IDX, split_data, normalize_features)


# Extract features and object ids form predicted detections
dataset_extractor = FeaturesExtractor()
try:
    det_features, det_object_ids = dataset_extractor.load_from_saved_pred(DETECTED_FEATURES_PATH, DETECTED_OBJECTS_IDS_PATH)
except FileNotFoundError:
    det_features, det_object_ids = dataset_extractor.extract_from_dataset_pred(DETECTIONS_PATH)
    dataset_extractor.save_pred(DETECTED_FEATURES_PATH, DETECTED_OBJECTS_IDS_PATH)

# Normalize features
det_features = normalize_features(det_features)

print('Shape of the extracted features:', det_features.shape)
print('Shape of the object ids:', det_object_ids.shape)


Shape of the extracted features: (48302, 7, 37)
Shape of the object ids: (48302,)


### Predictions processing functions

In [2]:
def get_classes(probabilities):
    """ Get classes from probabilities
    
    :param probabilities: array of probabilities in one-hot encoding
    
    :return: array of class indices
    """
    classes = np.argmax(probabilities, axis=1)
    return classes

def vote_classes(probabilities, object_ids):
    """ Vote classes for each object id
    
    :param probabilities: array of probabilities in one-hot encoding
    :param object_ids: array of object ids
    
    :return: array of class indices
    """
    objects_classes_probabilities = []
    for object_id in np.unique(object_ids):
        classes_probabilities = probabilities[det_object_ids == object_id]
        classes_probabilities = np.sum(classes_probabilities, axis=0)
        classes_probabilities = classes_probabilities/classes_probabilities.sum()
        objects_classes_probabilities.append((object_id, classes_probabilities))

    objects_classes_probabilities_dict = {object_id: classes_probabilities for object_id, classes_probabilities in objects_classes_probabilities}

    grouped_probabilities = np.array([objects_classes_probabilities_dict[id] for id in det_object_ids])

    voted_classes = get_classes(grouped_probabilities)
    return voted_classes


### Evaluation functions

In [3]:
def iou_confusion_matrix(gt, pred, ious, iou_threshold=0.5):
    """ Compute confusion matrix based on iou
    Last row  is for false positive detections under iou threshold
    
    :param gt: array of ground truth classes
    :param pred: array of predicted classes
    :param ious: array of ious
    :param iou_threshold: iou threshold for false positive detections

    :return: confusion matrix
    """
    classes = np.unique(gt)
    matrix = np.zeros((len(classes) + 1, len(classes)), dtype=np.int32)
    
    for i, gt_class in enumerate(classes):
        # Compute false positive detections under iou threshold
        mask = np.logical_and((ious < iou_threshold), (gt == gt_class))

        # Add false positive detections to last row of confusion matrix
        matrix[-1, i] = np.sum(mask)

        # Remove IOU false positive detections from gt and pred
        indexes = np.where(~mask)[0]
        gt = gt[indexes]
        pred = pred[indexes]
        ious = ious[indexes]

    # Compute confusion matrix
    for i, gt_class in enumerate(classes):
        for j, pred_class in enumerate(classes):
            matrix[i, j] = np.sum((gt == gt_class) & (pred == pred_class))

    return matrix

def get_metrics(cm, verbose=True):
    """ Compute metrics from confusion matrix

    :param cm: confusion matrix

    :return: accuracy, macro_precision, macro_recall, macro_f1_score
    """
    precisions = []
    recalls = []
    f1_scores = []
    num_classes = cm.shape[1]
    for i in range(num_classes):
                    TP = cm[i, i]
                    FP = np.sum(cm[:, i]) - TP
                    FN = np.sum(cm[i, :]) - TP
                    precision = TP / (TP + FP)
                    recall = TP / (TP + FN)
                    f1_score = 2 * precision * recall / (precision + recall)
                    precisions.append(precision)
                    recalls.append(recall)
                    f1_scores.append(f1_score)

    # Compute metrics over all classes
    accuracy = np.sum(np.diag(cm)) / np.sum(cm)
    macro_precision = np.mean(precisions)
    macro_recall = np.mean(recalls)
    macro_f1_score = np.mean(f1_scores)

    if verbose:
        print(f"Accuracy:   {accuracy*100:.2f} %")
        print(f"Precision:  {macro_precision*100:.2f} %")
        print(f"Recall:     {macro_recall*100:.2f} %")
        print(f"F1-score:   {macro_f1_score*100:.2f} %")

    return accuracy, macro_precision, macro_recall, macro_f1_score

def print_iou_metrics(gt, pred, ious):
    """ Print metrics from confusion matrix based on iou
    
    :param gt: array of ground truth classes
    :param pred: array of predicted classes
    :param ious: array of ious
    """
    cm = iou_confusion_matrix(gt, pred, ious)
    print('Confusion_matrix:')
    print(cm)
    print('\nMetrics:')
    get_metrics(cm)

    

## Evaluation of the 5 class model

### Load model an predict classes
Tesnorflow warnings are caused due to the use of recurrent dropout in the LSTM. It just informs us, that slower kernel has to be used.

In [24]:
MODEL_PATH = str(DATA_PATH / 'training' / 'models' / '5class_ensemble')
# Load model and predict classes from features
classif_ensemble_5 = Ensemble_ovo_ova(classes=list(range(5)))
classif_ensemble_5.load_models(MODEL_PATH)

cls_probabilities_5 = classif_ensemble_5.predict(det_features)
print('Class probabilities shape:', cls_probabilities_5.shape)

Class probabilities shape: (48302, 5)


### Get predicted classes on each frame and classes voted over object frames

In [5]:
single_clases_5 = get_classes(cls_probabilities_5)
tracked_classes_5 = vote_classes(cls_probabilities_5, det_object_ids)
gt_classes_5 = dataset_extractor.load_gt_classes(DETECTIONS_GT_CLASSES_PATH)
ious = np.load(DETECTIONS_IOU_PATH)

# Select test data
_, gt_classes_5_test = split_data(gt_classes_5, DATASET_SPLIT_IDX)
_, ious_test = split_data(ious, DATASET_SPLIT_IDX)
_, single_clases_5_test = split_data(single_clases_5, DATASET_SPLIT_IDX)
_, tracked_classes_5_test = split_data(tracked_classes_5, DATASET_SPLIT_IDX)

print('Shape of the ground truth classes:', gt_classes_5_test.shape)
print('Shape of the object ious:', ious_test.shape)
print('Shape of the single pred. classes:', single_clases_5_test.shape)
print('Shape of the tracked pred. classes:', tracked_classes_5_test.shape)

Shape of the ground truth classes: (9660,)
Shape of the object ious: (9660,)
Shape of the single pred. classes: (9660,)
Shape of the tracked pred. classes: (9660,)


### Evaluate model

In [6]:
print("Single classes - Test data")
print_iou_metrics(gt_classes_5_test, single_clases_5_test, ious_test)
print()
print("Grouped classes - Test data")
print_iou_metrics(gt_classes_5_test, tracked_classes_5_test, ious_test)

Single classes - Test data
Confusion_matrix:
[[ 317  158   12   37    5]
 [ 168 3961  473   31  196]
 [  60  354  805   88   82]
 [  23   12  101  404  166]
 [  41   29   29  219 1686]
 [  40   13    1   31  118]]

Metrics:
Accuracy:   74.25 %
Precision:  63.54 %
Recall:     68.25 %
F1-score:   65.66 %

Grouped classes - Test data
Confusion_matrix:
[[ 312  182    0   22   13]
 [  84 4530   34    0  181]
 [   0  280 1003   22   84]
 [   0    0  138  471   97]
 [   0    0    0    0 2004]
 [  40   13    1   31  118]]

Metrics:
Accuracy:   86.13 %
Precision:  82.78 %
Recall:     78.34 %
F1-score:   79.86 %


## Evaluation of the 3 class model
Tesnorflow warnings are caused due to the use of recurrent dropout in the LSTM. It just informs us, that slower kernel has to be used.

In [23]:
MODEL_PATH = str(DATA_PATH / 'training' / 'models' / '3class_ensemble')
# Load model and predict classes from features
classif_ensemble_3 = Ensemble_ovo_ova(classes=list(range(3)))
classif_ensemble_3.load_models(MODEL_PATH)

cls_probabilities_3 = classif_ensemble_3.predict(det_features)
print('Class probabilities shape:', cls_probabilities_3.shape)

Class probabilities shape: (48302, 3)


### Get predicted classes on each frame and classes voted over object frames

In [8]:
single_clases_3 = get_classes(cls_probabilities_3)
tracked_classes_3 = vote_classes(cls_probabilities_3, det_object_ids)
gt_classes_3 = dataset_extractor.load_gt_classes(DETECTIONS_GT_CLASSES_PATH, join_classes=True)
ious = np.load(DETECTIONS_IOU_PATH)

# Select test data
_, gt_classes_3_test = split_data(gt_classes_3, DATASET_SPLIT_IDX)
_, ious_test = split_data(ious, DATASET_SPLIT_IDX)
_, single_clases_3_test = split_data(single_clases_3, DATASET_SPLIT_IDX)
_, tracked_classes_3_test = split_data(tracked_classes_3, DATASET_SPLIT_IDX)

print('Shape of the ground truth classes:', gt_classes_3_test.shape)
print('Shape of the object ious:', ious_test.shape)
print('Shape of the single pred. classes:', single_clases_3_test.shape)
print('Shape of the tracked pred. classes:', tracked_classes_3_test.shape)

Shape of the ground truth classes: (9660,)
Shape of the object ious: (9660,)
Shape of the single pred. classes: (9660,)
Shape of the tracked pred. classes: (9660,)


### Evaluate model

In [9]:
print("Single classes - Test data")
print_iou_metrics(gt_classes_3_test, single_clases_3_test, ious_test)
print()
print("Grouped classes - Test data")
print_iou_metrics(gt_classes_3_test, tracked_classes_3_test, ious_test)

Single classes - Test data
Confusion_matrix:
[[ 344  144   41]
 [ 286 5490  442]
 [  74  121 2515]
 [  40   14  149]]

Metrics:
Accuracy:   86.43 %
Precision:  73.77 %
Recall:     82.04 %
F1-score:   77.17 %

Grouped classes - Test data
Confusion_matrix:
[[ 312  152   65]
 [  94 5839  285]
 [   0   77 2633]
 [  40   14  149]]

Metrics:
Accuracy:   90.93 %
Precision:  83.34 %
Recall:     83.35 %
F1-score:   83.03 %


### Create numpy files with prediction results for visualisation

In [20]:
import os

def generate_results(dataset_path, output_path, classes, ious):
    dataset_filepaths = sorted([os.path.join(dataset_path, file) for file in os.listdir(dataset_path)])

    print("Generating results in directory:", output_path)
    curr_object_ptr = 0
    for i, filepath in enumerate(dataset_filepaths, start=1):

        frame = np.load(filepath)
        frame_column_classes = np.full((frame.shape[0], 1),  -1)
        frame_column_ious = np.full((frame.shape[0], 1),  -1.0)

        frame_objects = np.unique(frame[:, COL_IDX["object_id"]])

        for object_id in frame_objects:
            object_rows_mask = frame[:, COL_IDX["object_id"]] == object_id

            if frame[object_rows_mask].shape[0] < 4:
                continue
            
            frame_column_classes[object_rows_mask, 0] = classes[curr_object_ptr]
            frame_column_ious[object_rows_mask, 0] = ious[curr_object_ptr]
            curr_object_ptr += 1

        result_frame = np.hstack((frame, frame_column_classes, frame_column_ious))
        save_filepath = os.path.join(output_path, os.path.basename(filepath))

        np.save(save_filepath, result_frame)

        print(f'Generating... {i} / {len(dataset_filepaths)}', end='\r')
    print(80*' ') # Clear line

### Generate results for each model

In [22]:
RESULTS_DIR = str(DATA_PATH / 'results')

OUT_PATH = str(DATA_PATH / 'results' / '5class_ensemble_single')
os.mkdir(OUT_PATH)
generate_results(DETECTIONS_PATH, OUT_PATH, single_clases_5, ious)

OUT_PATH = str(DATA_PATH / 'results' / '5class_ensemble_tracked')
os.mkdir(OUT_PATH)
generate_results(DETECTIONS_PATH, OUT_PATH, tracked_classes_5, ious)

OUT_PATH = str(DATA_PATH / 'results' / '3class_ensemble_single')
os.mkdir(OUT_PATH)
generate_results(DETECTIONS_PATH, OUT_PATH, single_clases_3, ious)

OUT_PATH = str(DATA_PATH / 'results' / '3class_ensemble_tracked')
os.mkdir(OUT_PATH)
generate_results(DETECTIONS_PATH, OUT_PATH, tracked_classes_3, ious)

Generating results in directory: /home/xgottw07/bp/data/results/5class_ensemble_single
                                                                                
Generating results in directory: /home/xgottw07/bp/data/results/5class_ensemble_tracked
                                                                                
Generating results in directory: /home/xgottw07/bp/data/results/3class_ensemble_single
                                                                                
Generating results in directory: /home/xgottw07/bp/data/results/3class_ensemble_tracked
                                                                                
